In [ ]:
import os
from database_tools.tools import BuildDatabase
from database_tools.utils import build_empty_database

# repo_dir = '/home/cam/Documents/database_tools/'
# os.chdir(repo_dir)

# build_empty_database()

In [ ]:
import os
from database_tools.tools import BuildDatabase
from database_tools.utils.compile import *

repo_dir = '/home/cam/Documents/database_tools/'
os.chdir(repo_dir)

worker = BuildDatabase(records_path='data/RECORDS-adults',
                       data_profile_csv='data/sample_count_data.csv',
                       max_records=200,
                       max_file_size=100,
                       data_dir='physionet.org/files/mimic3wdb/1.0/',
                       output_dir='data/mimic3/')
worker.run()

In [ ]:
from database_tools.utils.compile import read_dataset

dataset = read_dataset('data/mimic3/')
example = dataset.take(1)
for feature in example:
    print(feature)